In [1]:
import os
import pyciemss
import torch
import pandas as pd
import numpy as np
from typing import Dict, List, Callable

import pyciemss.visuals.plots as plots
import pyciemss.visuals.vega as vega
import pyciemss.visuals.trajectories as trajectories

from pyciemss.integration_utils.intervention_builder import (
    param_value_objective,
    start_time_objective,
)

smoke_test = ('CI' in os.environ)

In [2]:
model = "sir_3strain_petrinet.json"

In [3]:
start_time = 0.0
end_time = 62.0
logging_step_size = 1.0
num_samples =  10

In [4]:

result = pyciemss.sample(model, end_time, logging_step_size, num_samples, start_time=start_time)
display(result['data'].head())

ERROR:root:
                ###############################

                There was an exception in pyciemss

                Error occured in function: sample

                Function docs : 
    Load a model from a file, compile it into a probabilistic program, and sample from it.

    Args:
        model_path_or_json: Union[str, Dict]
            - A path to a AMR model file or JSON containing a model in AMR form.
        end_time: float
            - The end time of the sampled simulation.
        logging_step_size: float
            - The step size to use for logging the trajectory.
        num_samples: int
            - The number of samples to draw from the model.
        solver_method: str
            - The method to use for solving the ODE. See torchdiffeq's `odeint` method for more details.
            - If performance is incredibly slow, we suggest using `euler` to debug.
              If using `euler` results in faster simulation, the issue is likely that the model is s

ValueError: The model derivative could not be evaluated. Please check the model definition. This could be due to to a missing state variable or parameter, or an error in the derivative definition.
                                Trace Shapes:
                                 Param Sites:
 numeric_initial_state_func$$$_nodes.0._value
 numeric_initial_state_func$$$_nodes.1._value
 numeric_initial_state_func$$$_nodes.2._value
 numeric_initial_state_func$$$_nodes.3._value
 numeric_initial_state_func$$$_nodes.7._value
 numeric_initial_state_func$$$_nodes.8._value
 numeric_initial_state_func$$$_nodes.9._value
numeric_initial_state_func$$$_nodes.10._value
numeric_initial_state_func$$$_nodes.11._value
numeric_initial_state_func$$$_nodes.12._value
numeric_initial_state_func$$$_nodes.13._value
numeric_initial_state_func$$$_nodes.14._value
numeric_initial_state_func$$$_nodes.15._value
numeric_initial_state_func$$$_nodes.16._value
numeric_initial_state_func$$$_nodes.17._value
numeric_initial_state